In [16]:
%pip install fastapi numpy shapely pydantic requests pydantic_settings

  Using cached pydantic_settings-2.2.1-py3-none-any.whl.metadata (3.1 kB)
Using cached pydantic_settings-2.2.1-py3-none-any.whl (13 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from fastapi import APIRouter, HTTPException
from services.v1.scan_driver import ScanDriver
from services.v1.insert_driver import InsertDriver
from models.models import Variable
import requests

URLOSRM = "https://routing.openstreetmap.de/routed-car/route/v1/driving/{lng_start_point},{lat_start_point};{lng_end_point},{lat_end_point}?overview=full&geometries=geojson&steps=true&generate_hints=false"

In [18]:
duration = 0
class RequestRoute:
    route = None
    block_scan_data = []
    coordinates = None
    insert_times = None
    steps = None
    api_scan_driver = None
    api_route = None
    distance = 0
    old_time = 0
    options = None
    models = Variable()
    conditions = models.CONDITION
    full_route = models.ROUTE

    def __init__(self, s_lat, s_lng, e_lat, e_lng):
        self.models.setStartLatLng(s_lat, s_lng)
        self.models.setEndLatLng(e_lat, e_lng)

    # def serve(self):
        # for name in self.conditions:
        #     if name == "route":
        #         if self.conditions[name] == "osrm":
        #             self.dynamic_route("osrm")
        #         else:
        #             self.dynamic_route("graph")
        #         self.full_route["geometries"]["duration"] = duration
        #     elif name == "scan":
        #         if self.conditions[name]:
        #             self.scan_route()
        #             self.full_route["geometries"]["duration"] = duration
        #             self.full_route["geometries"]["blocks_scan"] = self.block_scan_data
        #         else:
        #             self.full_route["geometries"]["blocks_scan"] = []
        
        # self.full_route["geometries"]["distance"] = self.distance
        # self.full_route["geometries"]["route"] = self.route
        # self.full_route["geometries"]["options"] = self.options
        # self.full_route["start_point"] = [self.route[0][0], self.route[0][1]]
        # self.full_route["end_point"] = [self.route[-1][0], self.route[-1][1]]
        # return self.full_route
    def serve(self):
        for name in self.conditions:
            print("Name??: ", name)
            if name == "route":
                if self.conditions[name] == "osrm":
                    self.dynamic_route("osrm")
                else:
                    self.dynamic_route("graph")
                self.full_route["geometries"]["duration"] = duration  
            elif name == "scan":
                if self.conditions[name]:
                    self.scan_route()
                    self.full_route["geometries"]["duration"] = duration
                    self.full_route["geometries"]["blocks_scan"] = self.block_scan_data
                else:
                    self.full_route["geometries"]["blocks_scan"] = []
        
        self.full_route["geometries"]["distance"] = self.distance  
        self.full_route["geometries"]["route"] = self.route  
        self.full_route["geometries"]["options"] = self.options  
        # self.full_route["start_point"] = [self.route[0][0], self.route[0][1]]
        # self.full_route["end_point"] = [self.route[-1][0], self.route[-1][1]]
        # self.full_route["start_point"] = eval('[{},{}]'.format(self.route[0][0], self.route[0][1]))
        # self.full_route["end_point"] = eval('[{},{}]'.format(self.route[len(self.route) - 1][0], self.route[len(self.route) - 1][1]))
        return self.full_route

    def condition(self, **kwargs):
        edit = kwargs
        for name in edit:
            if edit[name] is not None:
                self.conditions[name] = edit[name]

    def dynamic_route(self, types):
        global duration
        if types is None or types == "osrm" or types != "graph":
            url = URLOSRM.format(lng_start_point=self.conditions["start_point"]["lng"], lat_start_point=self.conditions["start_point"]["lat"], lng_end_point=self.conditions["end_point"]["lng"], lat_end_point=self.conditions["end_point"]["lat"])
            osrm_bike = requests.get(url).json()
            duration = osrm_bike['routes'][0]['duration']
            self.distance = osrm_bike['routes'][0]['distance']
            self.old_time = osrm_bike['routes'][0]['duration']
            self.json_data = osrm_bike
            self.steps = osrm_bike['routes']
            self.coordinates = osrm_bike['routes'][0]['geometry']['coordinates']
           
            self.options_osrm()
        elif types == "graph":
            key = settings.KEY_GRAPH
            rand_key = random.choice(key)
            graph_bike = requests.get(
                'https://graphhopper.com/api/1//route?point={}%2C{}&point={}%2C{}&type=json&locale=en-US&vehicle=car&weighting=fastest&elevation=true&key={}&points_encoded=false'.format(
                    self.conditions["start_point"]["lat"], self.conditions["start_point"]["lng"], self.conditions["end_point"]["lat"], self.conditions["end_point"]["lng"], rand_key)).json()
            duration = graph_bike['paths'][0]['time']
            self.distance = graph_bike['paths'][0]['distance']
            self.old_time = graph_bike['paths'][0]['time']
            self.steps = graph_bike['paths']
            self.coordinates = graph_bike['paths'][0]['points']['coordinates']
            self.json_data = graph_bike
            self.options_graph()

        # print(['dynamic_route',self.conditions,osrm_bike])
        
        if not self.coordinates is None:
            return self.get_single_map()
        else:
            return self.dynamic_route(types)

    def scan_route(self):
        global duration
        scan = ScanDriver()
        insert = InsertDriver()
        drivers = insert.fake()
        get = scan.scan_driver(self.api_route, drivers)
        duration = get[0]
        self.block_scan_data = get[1]
        return self.block_scan_data

    def get_single_map(self):
        final_route = []
        coordinate_route = []
        lat = None
        lng = None
        for coordinate in self.coordinates:
            lng = coordinate[0]
            lat = coordinate[1]
            coordinate_route.append([lat, lng])
        final_route.append({"point1": [self.coordinates[0][1], self.coordinates[0][0]], "point2": [lat, lng], "route": coordinate_route, "duration": duration, "distance": self.distance})
        self.api_route = final_route[0]
        self.route = coordinate_route
        return self.route

    def get_multi_map(self):
        index_route = []
        final_route = []
        route = self.coordinates
        for i in range(0, len(self.json_data['routes'])):
            road = []
            for co in self.json_data['routes'][i]:
                coor = [co[1], co[0]]
                road.append(coor)
            index_route.append({"route": road, "duration": duration, "distance": self.distance})
        self.co_route = {"point1": [self.coordinates[0][1], self.coordinates[0][0]], "point2": [self.coordinates[len(self.coordinates) - 1][1], self.coordinates[len(self.coordinates) - 1][0]], "route": index_route}
        return self.co_route

    def options_osrm(self):
        options = []
        step = self.json_data['routes'][0]['legs'][0]['steps']
        for steps in step:
            point = [steps['maneuver']['location'][1], steps['maneuver']['location'][0]]
            distance = steps['distance']
            streetName = steps.get('name', "None")
            if "modifier" in steps['maneuver']:
                direction = "TURN_" + steps['maneuver']['modifier'].upper()
            else:
                direction = "NONE"
            options.append({"point": point, "distance": distance, "direction": direction, "streetName": streetName})
        self.options = options
        return self.options

    def options_graph(self):
        options = []
        step = self.json_data['paths'][0]['instructions']
        for instructions in step:
            distance = instructions['distance']
            direction = instructions['text']
            streetName = instructions['street_name']
            if not streetName:
                streetName = "None"
            options.append({"point": None, "distance": distance, "direction": direction, "streetName": streetName})
        self.options = options
        return self.options

In [19]:
req = RequestRoute(11.58609373356068,104.90691799652784,11.61390145589924,104.89831107937815)

In [20]:
results = req.serve()

Name??:  start_point
Name??:  end_point
Name??:  route
Name??:  weather
Name??:  scan
Name??:  traffic


In [8]:
results

{'code': 200,
 'start_point': [],
 'end_point': [],
 'geometries': {'route': [[11.586094, 104.906918],
   [11.585788, 104.906811],
   [11.585009, 104.906499],
   [11.583875, 104.906136],
   [11.584037, 104.905668],
   [11.584342, 104.904779],
   [11.585029, 104.902794],
   [11.58506, 104.902706],
   [11.585099, 104.902597],
   [11.585301, 104.902036],
   [11.585414, 104.901723],
   [11.585675, 104.900967],
   [11.585697, 104.900871],
   [11.5857, 104.900807],
   [11.585673, 104.900736],
   [11.585617, 104.900659],
   [11.585598, 104.90063],
   [11.585587, 104.900605],
   [11.58558, 104.900582],
   [11.585575, 104.900554],
   [11.585575, 104.900534],
   [11.585577, 104.900514],
   [11.585581, 104.900493],
   [11.585587, 104.900471],
   [11.585595, 104.900452],
   [11.585603, 104.900441],
   [11.585622, 104.900424],
   [11.585642, 104.900412],
   [11.585673, 104.900403],
   [11.585727, 104.900399],
   [11.586049, 104.900402],
   [11.586162, 104.900391],
   [11.586251, 104.900359],
   [11

In [9]:
res = req.dynamic_route(types='osrm')

In [11]:
req.json_data

{'code': 'Ok',
 'routes': [{'geometry': {'coordinates': [[104.906918, 11.586094],
     [104.906811, 11.585788],
     [104.906499, 11.585009],
     [104.906136, 11.583875],
     [104.905668, 11.584037],
     [104.904779, 11.584342],
     [104.902794, 11.585029],
     [104.902706, 11.58506],
     [104.902597, 11.585099],
     [104.902036, 11.585301],
     [104.901723, 11.585414],
     [104.900967, 11.585675],
     [104.900871, 11.585697],
     [104.900807, 11.5857],
     [104.900736, 11.585673],
     [104.900659, 11.585617],
     [104.90063, 11.585598],
     [104.900605, 11.585587],
     [104.900582, 11.58558],
     [104.900554, 11.585575],
     [104.900534, 11.585575],
     [104.900514, 11.585577],
     [104.900493, 11.585581],
     [104.900471, 11.585587],
     [104.900452, 11.585595],
     [104.900441, 11.585603],
     [104.900424, 11.585622],
     [104.900412, 11.585642],
     [104.900403, 11.585673],
     [104.900399, 11.585727],
     [104.900402, 11.586049],
     [104.900391, 11.58

In [12]:
option = req.options_osrm()

In [13]:
option

[{'point': [11.586094, 104.906918],
  'distance': 260,
  'direction': 'NONE',
  'streetName': 'ផ្លូវ ១០៥'},
 {'point': [11.583875, 104.906136],
  'distance': 646.9,
  'direction': 'TURN_RIGHT',
  'streetName': 'ផ្លូវ ២៧៣'},
 {'point': [11.585575, 104.900554],
  'distance': 715.7,
  'direction': 'TURN_RIGHT',
  'streetName': 'ផ្លូវ ៣៥៥'},
 {'point': [11.590922, 104.897102],
  'distance': 2510.9,
  'direction': 'TURN_RIGHT',
  'streetName': 'ផ្លូវ ជាសុផារ៉ា (ផ្លូវ ៥៩៨)'},
 {'point': [11.611323, 104.896301],
  'distance': 379.1,
  'direction': 'TURN_RIGHT',
  'streetName': ''},
 {'point': [11.612855, 104.899406],
  'distance': 165.9,
  'direction': 'TURN_LEFT',
  'streetName': ''},
 {'point': [11.613815, 104.898237],
  'distance': 0,
  'direction': 'TURN_RIGHT',
  'streetName': ''}]

In [14]:
multi = req.get_multi_map()

In [15]:
multi

{'point1': [11.586094, 104.906918],
 'point2': [11.613815, 104.898237],
 'route': [{'route': [['e', 'g'],
    ['e', 'l'],
    ['e', 'w'],
    ['e', 'w'],
    ['u', 'd'],
    ['i', 'd']],
   'duration': 344.6,
   'distance': 4678.4}]}